<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/TPU_AI_DEMODEC2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import jax
import jax.numpy as jnp
from jax import jit, random
from flax import linen as nn
import time
import json
import logging
from typing import Any, Tuple
import os
import shutil

# --- Configuration ---
DTYPE = jnp.bfloat16
D_MODEL = 512
SEQ_LEN = 64
BATCH_SIZE = 1
# Directory where the TPU profile data will be saved
TRACE_OUTPUT_DIR = "/tmp/tpu_agent_trace_output"

# Setup logging
logging.basicConfig(level=logging.INFO)

# --- 🖥️ TPU Resource Logging ---

def log_tpu_details():
    """Checks and logs the details of the available JAX devices (TPUs)."""
    devices = jax.devices()
    if not devices:
        logging.error("No JAX devices found. Ensure the TPU is configured correctly.")
        return

    logging.info("--- 🖥️ TPU RESOURCE DETAILS ---")
    logging.info(f"Total JAX Devices (Units/Cores): {len(devices)}")

    first_device = devices[0]
    logging.info(f"Device Kind (TPU Type): {first_device.device_kind}")
    logging.info("-" * 35)

# --- 1. TPU-Accelerated LLM Core (using JAX/Flax) ---

class TinyLLM(nn.Module):
    """A simplified Transformer block for demonstration, optimized for TPU."""
    d_model: int = D_MODEL

    @nn.compact
    def __call__(self, x):
        w_q = self.param('w_q', nn.initializers.glorot_uniform(), (self.d_model, self.d_model), DTYPE)
        q = jnp.dot(x, w_q)

        w1 = self.param('w1', nn.initializers.glorot_uniform(), (self.d_model, self.d_model * 4), DTYPE)
        b1 = self.param('b1', nn.initializers.zeros, (self.d_model * 4,), DTYPE)

        h = nn.gelu(jnp.dot(q, w1) + b1)

        w2 = self.param('w2', nn.initializers.glorot_uniform(), (self.d_model * 4, self.d_model), DTYPE)
        b2 = self.param('b2', nn.initializers.zeros, (self.d_model,), DTYPE)
        output = jnp.dot(h, w2) + b2

        return output

@jit
def llm_inference(params: Any, inputs: jnp.ndarray) -> Tuple[jnp.ndarray, jnp.ndarray]:
    """Performs one step of LLM inference on the TPU."""
    model = TinyLLM()
    raw_output = model.apply({'params': params}, inputs)

    tool_score = jnp.mean(raw_output) * 10.0

    return tool_score, raw_output

# --- 2. Agentic Tool (Run on Host CPU) ---

def execute_external_api_call(city: str) -> str:
    """A tool to simulate fetching real-time weather data (runs on host CPU)."""
    logging.info(f"Tool executing on CPU: Fetching data for {city}...")
    time.sleep(0.5)

    if "london" in city.lower():
        result_data = {"city": "London", "temp_c": 12, "condition": "Cloudy", "source": "API"}
    else:
        result_data = {"error": "City data not found", "source": "API"}

    return json.dumps(result_data)

# --- 3. Agent Orchestration ---

def run_agent(prompt: str, params: Any) -> Tuple[str, jnp.ndarray]:
    input_tensor = jnp.ones((BATCH_SIZE, SEQ_LEN, D_MODEL), dtype=DTYPE)

    logging.info("\n[1] Running initial LLM reasoning on TPU...")
    tool_score, _ = llm_inference(params, input_tensor)
    tool_score_val = tool_score.item()

    # MODIFIED THRESHOLD: Ensures tool use for the demo
    if tool_score_val > -2.0:
        tool_name = "execute_external_api_call"
        tool_args = {"city": "London"}

        logging.info(f"\n[2] LLM decision (TPU Score {tool_score_val:.4f}): USE TOOL: {tool_name}")

        tool_result_json = execute_external_api_call(tool_args["city"])

        logging.info(f"\n[3] Tool Result received. Rerunning LLM on TPU to generate final answer...")

        context_tensor = jnp.ones((BATCH_SIZE, SEQ_LEN, D_MODEL), dtype=DTYPE) * 0.2
        new_input_tensor = input_tensor + context_tensor
        final_score, _ = llm_inference(params, new_input_tensor)

        output_string = (f"Final Answer (TPU-Enhanced):\n"
                         f"Prompt: '{prompt}'\n"
                         f"Tool Output: {tool_result_json}\n"
                         f"Model Synthesis Score: {final_score.item():.4f}\n"
                         f"Conclusion: The agent used the weather tool and synthesized the final result based on the data.")
        return output_string, final_score

    else:
        logging.info(f"\n[2] LLM decision (TPU Score {tool_score_val:.4f}): NO TOOL NEEDED.")
        output_string = (f"Final Answer (Direct):\n"
                         f"Prompt: '{prompt}'\n"
                         f"Model Synthesis Score: {tool_score_val:.4f}\n"
                         f"Conclusion: The agent provided a direct answer without external tool use.")
        return output_string, tool_score


# --- Execution Block ---

if __name__ == "__main__":

    # --- STEP 0: Log Resources ---
    log_tpu_details()

    # Initialize the LLM parameters
    key = random.PRNGKey(42)
    init_rng, params_rng = random.split(key)

    model = TinyLLM(d_model=D_MODEL)
    params = model.init({'params': params_rng},
                        jnp.ones((BATCH_SIZE, SEQ_LEN, D_MODEL), dtype=DTYPE))['params']

    # Clean up previous trace directory and create a new one
    if os.path.exists(TRACE_OUTPUT_DIR):
        shutil.rmtree(TRACE_OUTPUT_DIR)
    os.makedirs(TRACE_OUTPUT_DIR, exist_ok=True)

    print("\n" + "="*70)
    print("🤖 Agentic AI Demo: TPU-Accelerated Loop")
    print("======================================================================")

    # --- STEP 4: Start JAX Profiler (Non-blocking) ---
    print(f"INFO: Starting JAX Profiler to capture trace data...")
    jax.profiler.start_trace(TRACE_OUTPUT_DIR)

    # Run the agent demo
    final_output_string, final_score_tensor = run_agent("I need the current weather in London to decide on my outfit.", params)

    # Block until the final computation is complete
    final_score_tensor.block_until_ready()

    # --- STEP 5: Stop JAX Profiler and Finalize ---
    jax.profiler.stop_trace()
    print(f"INFO: JAX Profiler trace saved successfully to directory: {TRACE_OUTPUT_DIR}")

    print("\n" + "="*70)
    print("AGENT DEMO RESULT SUMMARY")
    print("======================================================================")
    print(final_output_string)
    print("======================================================================")


🤖 Agentic AI Demo: TPU-Accelerated Loop
INFO: Starting JAX Profiler to capture trace data...
INFO: JAX Profiler trace saved successfully to directory: /tmp/tpu_agent_trace_output

AGENT DEMO RESULT SUMMARY
Final Answer (TPU-Enhanced):
Prompt: 'I need the current weather in London to decide on my outfit.'
Tool Output: {"city": "London", "temp_c": 12, "condition": "Cloudy", "source": "API"}
Model Synthesis Score: -1.6875
Conclusion: The agent used the weather tool and synthesized the final result based on the data.


In [ ]:
import jax
import jax.numpy as jnp
from jax import jit, random
from flax import linen as nn
from jax.sharding import NamedSharding, PartitionSpec as P
import time
import json
import logging
from typing import Any, Tuple

# --- Configuration ---
DTYPE = jnp.bfloat16
D_MODEL = 512
SEQ_LEN = 64
BATCH_SIZE = 1

# Setup logging
logging.basicConfig(level=logging.INFO)

# --- 🖥️ TPU Resource Logging ---
def log_tpu_details():
    """Checks and logs the details of the available JAX devices (TPUs), including specific type info."""
    devices = jax.devices()
    if not devices:
        logging.error("No JAX devices found. Ensure the TPU is configured correctly.")
        return

    logging.info("--- 🖥️ TPU RESOURCE DETAILS ---")
    first_device = devices[0]
    logging.info(f"Total JAX Devices (Units/Cores): {len(devices)}")
    logging.info(f"Device Kind: {first_device.device_kind}")
    logging.info(f"Full Device Description: {repr(first_device)}")
    logging.info("-" * 35)

# --- 1. TPU-Accelerated LLM Core (using JAX/Flax) ---

class TinyLLM(nn.Module):
    d_model: int = D_MODEL

    @nn.compact
    def __call__(self, x):
        w_q = self.param('w_q', nn.initializers.glorot_uniform(), (self.d_model, self.d_model), DTYPE)
        q = jnp.dot(x, w_q)
        w1 = self.param('w1', nn.initializers.glorot_uniform(), (self.d_model, self.d_model * 4), DTYPE)
        b1 = self.param('b1', nn.initializers.zeros, (self.d_model * 4,), DTYPE)
        h = nn.gelu(jnp.dot(q, w1) + b1)
        w2 = self.param('w2', nn.initializers.glorot_uniform(), (self.d_model * 4, self.d_model), DTYPE)
        b2 = self.param('b2', nn.initializers.zeros, (self.d_model,), DTYPE)
        output = jnp.dot(h, w2) + b2
        return output

@jit
def llm_inference(params: Any, inputs: jnp.ndarray) -> Tuple[jnp.ndarray, jnp.ndarray]:
    model = TinyLLM()
    raw_output = model.apply({'params': params}, inputs)
    tool_score = jnp.mean(raw_output) * 10.0
    return tool_score, raw_output

# --- 2. Agentic Tool (Run on Host CPU) ---

def execute_external_api_call(city: str) -> str:
    logging.info(f"Tool executing on CPU: Fetching data for {city}...")
    time.sleep(0.5)
    if "london" in city.lower():
        result_data = {"city": "London", "temp_c": 12, "condition": "Cloudy", "source": "API"}
    else:
        result_data = {"error": "City data not found", "source": "API"}
    return json.dumps(result_data)

# --- 3. Agent Orchestration ---

def run_agent(prompt: str, params: Any) -> str:
    input_tensor = jnp.ones((BATCH_SIZE, SEQ_LEN, D_MODEL), dtype=DTYPE)

    logging.info("\n[A1] Running initial LLM reasoning on TPU...")
    tool_score, _ = llm_inference(params, input_tensor)
    tool_score_val = tool_score.item()

    if tool_score_val > -2.0:
        tool_name = "execute_external_api_call"
        tool_args = {"city": "London"}

        logging.info(f"\n[A2] LLM decision (TPU Score {tool_score_val:.4f}): USE TOOL: {tool_name}")

        tool_result_json = execute_external_api_call(tool_args["city"])

        logging.info(f"\n[A3] Tool Result received. Rerunning LLM on TPU to generate final answer...")

        context_tensor = jnp.ones((BATCH_SIZE, SEQ_LEN, D_MODEL), dtype=DTYPE) * 0.2
        new_input_tensor = input_tensor + context_tensor
        final_score, _ = llm_inference(params, new_input_tensor)

        return (f"Final Answer (TPU-Enhanced):\n"
                f"Prompt: '{prompt}'\n"
                f"Tool Output: {tool_result_json}\n"
                f"Model Synthesis Score: {final_score.item():.4f}\n"
                f"Conclusion: The agent used the weather tool and synthesized the final result based on the data.")

    else:
        logging.info(f"\n[A2] LLM decision (TPU Score {tool_score_val:.4f}): NO TOOL NEEDED.")
        return (f"Final Answer (Direct):\n"
                f"Prompt: '{prompt}'\n"
                f"Model Synthesis Score: {tool_score_val:.4f}\n"
                f"Conclusion: The agent provided a direct answer without external tool use.")


# --- Execution Block ---

if __name__ == "__main__":

    # --- STEP 0: Log Resources ---
    log_tpu_details()

    # --- JAX SHARDING INTEGRATION ---
    print("\n" + "#"*70)
    print("✨ JAX SHARDING DEMONSTRATION (Single Device Setup)")
    print("#"*70)

    device_count = jax.device_count()
    print(f"INFO: Found {device_count} devices for sharding setup.")

    # --- 1. Corrected Mesh Creation ---
    print("--- 1. Creating the Device Mesh (1x1 Logical Grid) ---")
    mesh = jax.make_mesh((device_count,), ('X',))
    print(f"Mesh created with axes: {mesh.axis_names}")
    print(f"Number of physical devices in mesh: {mesh.size}")

    # --- 2. Create the Array ---
    key = jax.random.key(0)
    x = jax.random.normal(key, (256, 256), dtype=jnp.float32)
    print(f"\n--- 2. Initial Array Created: {x.shape} ({x.dtype}) ---")

    # --- 3. Define the Sharding Specification ---
    sharding_spec = P('X', None)
    sharding_rule = NamedSharding(mesh, sharding_spec)
    print(f"--- 3. Sharding Rule Defined: Partitioning {sharding_spec} ---")

    # --- 4. Distribute the Array ---
    y = jax.device_put(x, sharding_rule)
    print(f"\n--- 4. Array Distributed ('y') ---")
    print(f"Resulting array shape: {y.shape}")
    print(f"Partition size per device: {y.addressable_shards[0].data.shape}")

    # --- 5. Visualize the Sharding ---
    print("\n--- 5. Visualizing Array Sharding ---")
    jax.debug.visualize_array_sharding(y)

    print("\n" + "#"*70)
    print("🤖 STARTING TPU AGENT DEMO EXECUTION")
    print("#"*70)

    # --- TPU AGENT EXECUTION ---

    # Initialize the LLM parameters
    key = random.PRNGKey(42)
    init_rng, params_rng = random.split(key)

    model = TinyLLM(d_model=D_MODEL)
    params = model.init({'params': params_rng},
                        jnp.ones((BATCH_SIZE, SEQ_LEN, D_MODEL), dtype=DTYPE))['params']

    print("\n" + "="*70)
    print("🤖 Agentic AI Demo: TPU-Accelerated Loop")
    print("="*70)

    # Run the agent demo
    result = run_agent("I need the current weather in London to decide on my outfit.", params)

    print("\n" + "="*70)
    print("AGENT DEMO RESULT SUMMARY")
    print("="*70)
    print(result)
    print("="*70)


######################################################################
✨ JAX SHARDING DEMONSTRATION (Single Device Setup)
######################################################################
INFO: Found 1 devices for sharding setup.
--- 1. Creating the Device Mesh (1x1 Logical Grid) ---
Mesh created with axes: ('X',)
Number of physical devices in mesh: 1

--- 2. Initial Array Created: (256, 256) (float32) ---
--- 3. Sharding Rule Defined: Partitioning PartitionSpec('X', None) ---

--- 4. Array Distributed ('y') ---
Resulting array shape: (256, 256)
Partition size per device: (256, 256)

--- 5. Visualizing Array Sharding ---


                         
                         
                         
                         
                         
          TPU 0          
                         
                         
                         
                         
                         


######################################################################
🤖 STARTING TPU AGENT DEMO EXECUTION
######################################################################

🤖 Agentic AI Demo: TPU-Accelerated Loop

AGENT DEMO RESULT SUMMARY
Final Answer (TPU-Enhanced):
Prompt: 'I need the current weather in London to decide on my outfit.'
Tool Output: {"city": "London", "temp_c": 12, "condition": "Cloudy", "source": "API"}
Model Synthesis Score: -1.6875
Conclusion: The agent used the weather tool and synthesized the final result based on the data.


In [ ]:
import jax
import jax.numpy as jnp
from google import genai
from typing import Any, Tuple
import time
import json
import logging
import os
from jax.sharding import NamedSharding, PartitionSpec as P
from google.colab import userdata
from google.genai import types

# --- WARNING: This code runs your client initialization first ---

# --- GEMINI CLIENT SETUP ---
try:
    # This block requires 'google.colab.userdata' to be available.
    GOOGLE_API_KEY = userdata.get('GEMINI')
    if not GOOGLE_API_KEY:
        raise ValueError("Error: 'GEMINI' API key not found in Colab Secrets.")

    client = genai.Client(api_key=GOOGLE_API_KEY)

    # Using the stable model for chat
    GEMINI_MODEL_ID = 'gemini-2.5-pro'

    print(f"Gemini client successfully configured for model: {GEMINI_MODEL_ID}.")

except Exception as e:
    print(f"Configuration Error: {e}")
    raise SystemExit("Exiting script due to configuration failure.")

# --- Agent Configuration ---
logging.basicConfig(level=logging.INFO)
tool_function_map = {
    "execute_external_api_call": lambda city: execute_external_api_call(city)
}

# --- 🖥️ TPU Resource Logging ---

def log_tpu_details():
    """Checks and logs the details of the available JAX devices (TPUs), including specific type info."""
    devices = jax.devices()
    if not devices:
        logging.error("No JAX devices found.")
        return
    logging.info("--- 🖥️ TPU RESOURCE DETAILS ---")
    first_device = devices[0]
    logging.info(f"Total JAX Devices (Units/Cores): {len(devices)}")
    logging.info(f"Device Kind: {first_device.device_kind}")
    logging.info(f"Full Device Description: {repr(first_device)}")
    logging.info("-" * 35)


# --- 1. Agentic Tool (Runs on Host CPU) ---

def execute_external_api_call(city: str) -> str:
    """A tool to simulate fetching real-time weather data. Runs on the Host CPU."""
    logging.info(f"Tool executing on CPU: Fetching data for {city}...")
    time.sleep(0.5)

    if "london" in city.lower():
        result_data = {"city": "London", "temp_c": 12, "condition": "Cloudy", "source": "API"}
    else:
        result_data = {"error": "City data not found", "source": "API"}

    return json.dumps(result_data)


# --- 2. Agent Orchestration (Using Gemini API with Chat) ---

def run_gemini_agent(prompt: str) -> str:

    tools_config = [execute_external_api_call]

    # Use AUTO mode, as Chat sessions handle the context needed for this decision well
    tool_config = types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode="AUTO")
    )

    # 1. Create a chat session with the tools attached
    chat = client.chats.create(
        model=GEMINI_MODEL_ID,
        config=types.GenerateContentConfig(
            tools=tools_config,
            tool_config=tool_config
        )
    )

    # 2. First Message: Ask Gemini for a decision (Reasoning)
    logging.info("\n[A1] Sending prompt to Gemini chat for initial reasoning and tool decision...")
    response = chat.send_message(prompt)

    # --- Check for Function Calls ---
    function_calls = response.candidates[0].function_calls if response.candidates and hasattr(response.candidates[0], 'function_calls') else None

    # 3. Check for Tool Call
    if function_calls:
        # Use the first function call suggested
        fc = function_calls[0]
        tool_name = fc.name
        tool_args = dict(fc.args)

        logging.info(f"\n[A2] Gemini decided to use Tool: {tool_name} with args: {tool_args}")

        # 4. Execute the tool locally (on the Host CPU)
        if tool_name in tool_function_map:
            tool_func = tool_function_map[tool_name]
            tool_result = tool_func(**tool_args)

            logging.info(f"Tool Result (CPU): {tool_result}")

            # 5. Second Message: Send result back to Gemini for synthesis (Chat handles context)
            logging.info("\n[A3] Sending Tool Result back to Gemini chat for final synthesis...")

            function_response_part = genai.types.Part.from_function_response(
                name=tool_name,
                response={"result": tool_result}
            )

            second_response = chat.send_message([function_response_part])

            # Ensure text extraction
            final_conclusion = second_response.text if second_response.text else "Model Synthesis Successful, Text Unavailable (SDK issue)."

            return (f"Final Answer (Gemini-Enhanced via Chat):\n"
                    f"Model: {GEMINI_MODEL_ID} (Chat Function Calling)\\n"\
                    f"Prompt: '{prompt}'\\n"\
                    f"Tool Output: {tool_result}\\n"\
                    f"Conclusion: {final_conclusion}")
        else:
            return f"Error: Agent tried to call unknown tool: {tool_name}"

    else:
        # 6. Direct Answer (Model decided to answer directly)
        logging.info("\n[A2] Gemini provided a direct answer without tool use.")
        direct_conclusion = response.text if response.text else "Model provided a structured response that lacks text."

        return (f"Final Answer (Direct Answer via Chat):\n"
                f"Model: {GEMINI_MODEL_ID}\\n"\
                f"Prompt: '{prompt}'\\n"\
                f"Conclusion: {direct_conclusion}")


# --- Execution Block ---

if __name__ == "__main__":

    # --- Log Resources ---
    log_tpu_details()

    # --- JAX SHARDING DEMONSTRATION (Your Code Integrated) ---
    print("\n" + "#"*70)
    print("✨ JAX SHARDING DEMONSTRATION (TPU Array Initialization)")
    print("#"*70)

    device_count = jax.device_count()
    print(f"INFO: Found {device_count} devices for sharding setup.")

    # 1. Mesh Creation (Adapted to available devices)
    print("--- 1. Creating the Device Mesh (1x1 Logical Grid) ---")
    mesh = jax.make_mesh((device_count,), ('X',))
    print(f"Mesh created with axes: {mesh.axis_names}")
    print(f"Number of physical devices in mesh: {mesh.size}")

    # 2. Create the Array (Using small array and float32 for simplicity)
    key = jax.random.key(0)
    x = jax.random.normal(key, (256, 256), dtype=jnp.float32)
    print(f"\n--- 2. Initial Array Created: {x.shape} ({x.dtype}) ---")

    # 3. Define the Sharding Specification (Partitions across the single 'X' axis)
    sharding_spec = P('X', None)
    sharding_rule = NamedSharding(mesh, sharding_spec)
    print(f"--- 3. Sharding Rule Defined: Partitioning {sharding_spec} ---")

    # 4. Distribute the Array
    y = jax.device_put(x, sharding_rule)
    print(f"\n--- 4. Array Distributed ('y') ---")
    print(f"Partition size per device: {y.addressable_shards[0].data.shape}")

    # 5. Visualize the Sharding
    print("\n--- 5. Visualizing Array Sharding ---")
    jax.debug.visualize_array_sharding(y)

    print("\n" + "#"*70)
    print("🤖 STARTING GEMINI API AGENT EXECUTION (CHAT FIXED)")
    print("#"*70)

    # --- GEMINI AGENT EXECUTION ---

    print("\n" + "="*70)
    print(f"🤖 Agentic AI Demo: Gemini Function Calling Loop")
    print("="*70)

    # Run the agent demo
    result = run_gemini_agent("I need the current weather in London to decide on my outfit.")

    print("\n" + "="*70)
    print("AGENT DEMO RESULT SUMMARY")
    print("="*70)
    print(result)
    print("="*70)

Gemini client successfully configured for model: gemini-2.5-pro.

######################################################################
✨ JAX SHARDING DEMONSTRATION (TPU Array Initialization)
######################################################################
INFO: Found 1 devices for sharding setup.
--- 1. Creating the Device Mesh (1x1 Logical Grid) ---
Mesh created with axes: ('X',)
Number of physical devices in mesh: 1

--- 2. Initial Array Created: (256, 256) (float32) ---
--- 3. Sharding Rule Defined: Partitioning PartitionSpec('X', None) ---

--- 4. Array Distributed ('y') ---
Partition size per device: (256, 256)

--- 5. Visualizing Array Sharding ---


                         
                         
                         
                         
                         
          TPU 0          
                         
                         
                         
                         
                         


######################################################################
🤖 STARTING GEMINI API AGENT EXECUTION (CHAT FIXED)
######################################################################

🤖 Agentic AI Demo: Gemini Function Calling Loop

AGENT DEMO RESULT SUMMARY
Final Answer (Direct Answer via Chat):
Model: gemini-2.5-pro\nPrompt: 'I need the current weather in London to decide on my outfit.'\nConclusion: The weather in London is currently cloudy with a temperature of 12 degrees Celsius.



In [ ]:
import jax
import jax.numpy as jnp
from google import genai
from typing import Any, Tuple
import time
import json
import logging
import os
from jax.sharding import NamedSharding, PartitionSpec as P
from google.colab import userdata
from google.genai import types

# --- WARNING: This code runs your client initialization first ---

# --- GEMINI CLIENT SETUP ---
try:
    # This block requires 'google.colab.userdata' to be available.
    GOOGLE_API_KEY = userdata.get('GEMINI')
    if not GOOGLE_API_KEY:
        raise ValueError("Error: 'GEMINI' API key not found in Colab Secrets.")

    client = genai.Client(api_key=GOOGLE_API_KEY)

    # Using the stable model for chat
    GEMINI_MODEL_ID = 'gemini-2.5-pro'

    print(f"Gemini client successfully configured for model: {GEMINI_MODEL_ID}.")

except Exception as e:
    print(f"Configuration Error: {e}")
    raise SystemExit("Exiting script due to configuration failure.")

# --- Agent Configuration ---
logging.basicConfig(level=logging.INFO)
tool_function_map = {
    "execute_external_api_call": lambda city: execute_external_api_call(city)
}

# --- 🖥️ TPU Resource Logging ---

def log_tpu_details():
    """Checks and logs the details of the available JAX devices (TPUs), including specific type info."""
    devices = jax.devices()
    if not devices:
        logging.error("No JAX devices found.")
        return
    logging.info("--- 🖥️ TPU RESOURCE DETAILS ---")
    first_device = devices[0]
    logging.info(f"Total JAX Devices (Units/Cores): {len(devices)}")
    logging.info(f"Device Kind: {first_device.device_kind}")
    logging.info(f"Full Device Description: {repr(first_device)}")
    logging.info("-" * 35)


# --- 1. Agentic Tool (Runs on Host CPU - MODIFIED to return Vostok data for specific demo) ---

def execute_external_api_call(city: str) -> str:
    """A tool to simulate fetching real-time weather data. Runs on the Host CPU."""
    logging.info(f"Tool executing on CPU: Fetching data for {city}...")
    time.sleep(0.5)

    # Force a unique, tool-dependent result to prove tool was used for the new prompt
    if "vostok" in city.lower():
        result_data = {"city": "Vostok Station, Antarctica", "temp_c": -60, "condition": "Severe Clear Cold", "source": "API"}
    else:
        # Fallback to original London data
        result_data = {"city": "London", "temp_c": 12, "condition": "Cloudy", "source": "API"}

    return json.dumps(result_data)


# --- 2. Agent Orchestration (Using Gemini API with Chat) ---

def run_gemini_agent(prompt: str) -> str:

    tools_config = [execute_external_api_call]

    # Use AUTO mode, as Chat sessions handle the context needed for this decision well
    tool_config = types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode="AUTO")
    )

    # 1. Create a chat session with the tools attached
    chat = client.chats.create(
        model=GEMINI_MODEL_ID,
        config=types.GenerateContentConfig(
            tools=tools_config,
            tool_config=tool_config
        )
    )

    # 2. First Message: Ask Gemini for a decision (Reasoning)
    logging.info("\n[A1] Sending prompt to Gemini chat for initial reasoning and tool decision...")
    response = chat.send_message(prompt)

    # --- Check for Function Calls ---
    function_calls = response.candidates[0].function_calls if response.candidates and hasattr(response.candidates[0], 'function_calls') else None

    # 3. Check for Tool Call
    if function_calls:
        # Use the first function call suggested
        fc = function_calls[0]
        tool_name = fc.name
        tool_args = dict(fc.args)

        logging.info(f"\n[A2] Gemini decided to use Tool: {tool_name} with args: {tool_args}")

        # 4. Execute the tool locally (on the Host CPU)
        if tool_name in tool_function_map:
            tool_func = tool_function_map[tool_name]
            # Extract city argument, defaulting to a test city if missing
            city_arg = tool_args.get("city", "London")
            tool_result = tool_func(city=city_arg)

            logging.info(f"Tool Result (CPU): {tool_result}")

            # 5. Second Message: Send result back to Gemini for synthesis (Chat handles context)
            logging.info("\n[A3] Sending Tool Result back to Gemini chat for final synthesis...")

            function_response_part = genai.types.Part.from_function_response(
                name=tool_name,
                response={"result": tool_result}
            )

            # Send the tool result back as a part of the chat history
            second_response = chat.send_message([function_response_part])

            # Ensure text extraction
            final_conclusion = second_response.text if second_response.text else "Model Synthesis Successful, Text Unavailable (SDK issue)."

            return (f"Final Answer (Gemini-Enhanced via Chat):\n"
                    f"Model: {GEMINI_MODEL_ID} (Chat Function Calling)\\n"\
                    f"Prompt: '{prompt}'\\n"\
                    f"Tool Output: {tool_result}\\n"\
                    f"Conclusion: {final_conclusion}")
        else:
            return f"Error: Agent tried to call unknown tool: {tool_name}"

    else:
        # 6. Direct Answer (Model decided to answer directly)
        logging.info("\n[A2] Gemini provided a direct answer without tool use.")
        direct_conclusion = response.text if response.text else "Model provided a structured response that lacks text."

        return (f"Final Answer (Direct Answer via Chat):\n"
                f"Model: {GEMINI_MODEL_ID}\\n"\
                f"Prompt: '{prompt}'\\n"\
                f"Conclusion: {direct_conclusion}")


# --- Execution Block ---

if __name__ == "__main__":

    # --- Log Resources ---
    log_tpu_details()

    # --- JAX SHARDING DEMONSTRATION (Your Code Integrated) ---
    print("\n" + "#"*70)
    print("✨ JAX SHARDING DEMONSTRATION (TPU Array Initialization)")
    print("#"*70)

    device_count = jax.device_count()
    print(f"INFO: Found {device_count} devices for sharding setup.")

    # 1. Mesh Creation (Adapted to available devices)
    print("--- 1. Creating the Device Mesh (1x1 Logical Grid) ---")
    mesh = jax.make_mesh((device_count,), ('X',))
    print(f"Mesh created with axes: {mesh.axis_names}")
    print(f"Number of physical devices in mesh: {mesh.size}")

    # 2. Create the Array (Using small array and float32 for simplicity)
    key = jax.random.key(0)
    x = jax.random.normal(key, (256, 256), dtype=jnp.float32)
    print(f"\n--- 2. Initial Array Created: {x.shape} ({x.dtype}) ---")

    # 3. Define the Sharding Specification (Partitions across the single 'X' axis)
    sharding_spec = P('X', None)
    sharding_rule = NamedSharding(mesh, sharding_spec)
    print(f"--- 3. Sharding Rule Defined: Partitioning {sharding_spec} ---")

    # 4. Distribute the Array
    y = jax.device_put(x, sharding_rule)
    print(f"\n--- 4. Array Distributed ('y') ---")
    print(f"Partition size per device: {y.addressable_shards[0].data.shape}")

    # 5. Visualize the Sharding
    print("\n--- 5. Visualizing Array Sharding ---")
    jax.debug.visualize_array_sharding(y)

    print("\n" + "#"*70)
    print("🤖 STARTING GEMINI API AGENT EXECUTION (CHAT FIXED, PROMPT MODIFIED)")
    print("#"*70)

    # --- GEMINI AGENT EXECUTION ---

    print("\n" + "="*70)
    print(f"🤖 Agentic AI Demo: Gemini Function Calling Loop")
    print("="*70)

    # Run the agent demo with the specific prompt
    # NEW PROMPT: Designed to force the model to call the tool because it's highly specific
    result = run_gemini_agent("What is the current temperature in Vostok Station, Antarctica? I need this exact temperature reading.")

    print("\n" + "="*70)
    print("AGENT DEMO RESULT SUMMARY")
    print("="*70)
    print(result)
    print("="*70)

Gemini client successfully configured for model: gemini-2.5-pro.

######################################################################
✨ JAX SHARDING DEMONSTRATION (TPU Array Initialization)
######################################################################
INFO: Found 1 devices for sharding setup.
--- 1. Creating the Device Mesh (1x1 Logical Grid) ---
Mesh created with axes: ('X',)
Number of physical devices in mesh: 1

--- 2. Initial Array Created: (256, 256) (float32) ---
--- 3. Sharding Rule Defined: Partitioning PartitionSpec('X', None) ---

--- 4. Array Distributed ('y') ---
Partition size per device: (256, 256)

--- 5. Visualizing Array Sharding ---


                         
                         
                         
                         
                         
          TPU 0          
                         
                         
                         
                         
                         


######################################################################
🤖 STARTING GEMINI API AGENT EXECUTION (CHAT FIXED, PROMPT MODIFIED)
######################################################################

🤖 Agentic AI Demo: Gemini Function Calling Loop

AGENT DEMO RESULT SUMMARY
Final Answer (Direct Answer via Chat):
Model: gemini-2.5-pro\nPrompt: 'What is the current temperature in Vostok Station, Antarctica? I need this exact temperature reading.'\nConclusion: I am sorry, but I cannot provide the exact temperature for Vostok Station, Antarctica, as my weather data is not that precise. It is a remote research station, and I can only provide weather information for larger cities.
